In [18]:
import psycopg2
from datetime import datetime
import psycopg2.extras 

host = "localhost"
database = "uirapurudb-dump-prod"
user = "postgres"
password = "Giogears99"

try:
    # Cria uma conexão com o banco de dados
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )

    # Cria um cursor para executar a consulta
    cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

    # Executa a consulta desejada
    query = """
    SELECT 
        CAST(MAX(((m.payload::jsonb)->>'pt')::NUMERIC) AS NUMERIC(10,2))  AS "Potência media(W)",
        reference_time AS "Hora"
    FROM 
	    messages m
    JOIN
    (
        SELECT 
        DISTINCT
        	date_trunc('hour', selected_time) AS reference_time,
            date_trunc('hour', selected_time) - INTERVAL '5 minutes' AS start_time,
            date_trunc('hour', selected_time) + INTERVAL '5 minutes' AS end_time
        FROM
            generate_series(
                '2023-09-01 00:00:00'::timestamp, -- Hora inicial
                '2023-09-01 18:59:00'::timestamp, -- Hora final
                '10 minutes'::interval -- Intervalo de iteração
            ) AS selected_time
    ) d
    ON 
    m.created_at >= d.start_time
    AND m.created_at <= d.end_time
    WHERE 
    device_id = 1
    GROUP BY
	d.reference_time
    ORDER BY reference_time ASC;
    """
    cursor.execute(query)

    lista = []
    # Recupera os resultados da consulta
    results = cursor.fetchall()
    # print(results)
    for linha in results:
        # print(linha)
        dicionario = {'data_hora':0, 'pot':0}
        for dado in linha:
            if (type(dado) == datetime):
                # print(dado.hour)
                dicionario['data_hora'] = datetime.strftime(dado,'%y-%m-%d %H:%M:%S')
            else:
                dicionario["pot"] = float(dado)
        lista.append(dicionario)
    print(lista)
    # for data in results:
    #     print(data)
    #     print(type(data))
    #     for data2 in data:
    #         print(data2)
    #         print(type(data2))
    
    # Fecha o cursor e a conexão com o banco de dados
    cursor.close()
    connection.close()

except psycopg2.Error as e:
    print("Erro ao conectar ao banco de dados:", e)

[{'data_hora': '23-09-01 05:00:00', 'pot': 138.0}, {'data_hora': '23-09-01 06:00:00', 'pot': 150.0}, {'data_hora': '23-09-01 07:00:00', 'pot': 4752.0}, {'data_hora': '23-09-01 08:00:00', 'pot': 21510.0}, {'data_hora': '23-09-01 09:00:00', 'pot': 53178.0}, {'data_hora': '23-09-01 10:00:00', 'pot': 77640.0}, {'data_hora': '23-09-01 11:00:00', 'pot': 83214.0}, {'data_hora': '23-09-01 12:00:00', 'pot': 95256.0}, {'data_hora': '23-09-01 13:00:00', 'pot': 84012.0}, {'data_hora': '23-09-01 14:00:00', 'pot': 95022.0}, {'data_hora': '23-09-01 15:00:00', 'pot': 87228.0}, {'data_hora': '23-09-01 16:00:00', 'pot': 53724.0}, {'data_hora': '23-09-01 17:00:00', 'pot': 15078.0}, {'data_hora': '23-09-01 18:00:00', 'pot': 1026.0}]


In [ ]:
import datetime

mytime = datetime.datetime(2023, 8, 27, 8, 0)

print(type(mytime))

print(mytime)

print(f'mytime hour = {mytime.hour}')

In [ ]:
import requests as req
import json
import psycopg2


def encontrar_chave(dicionario, chave):
    if isinstance(dicionario, dict):  # Verifica se é um dicionário
        if chave in dicionario:  # Verifica se a chave está no dicionário atual
            return dicionario[chave]
        else:
            for valor in dicionario.values():  # Percorre os valores do dicionário
                resultado = encontrar_chave(valor, chave)
                if resultado is not None:
                    return resultado
    elif isinstance(dicionario, list):  # Verifica se é uma lista
        for item in dicionario:  # Percorre os itens da lista
            resultado = encontrar_chave(item, chave)
            if resultado is not None:
                return resultado
    return None  # Retorna None se a chave não for encontrada


flag = False  # para nao ficar requisitando da api em todo teste

# formato requerido URL : https://re.jrc.ec.europa.eu/api/v5_2/DRcalc?lat=-1.0714&lon=-48.1264&month=9&global=1&raddatabase=PVGIS-SARAH2&localtime=1&angle=10&aspect=180&showtemperatures=1&outputformat=json

api_url = 'https://re.jrc.ec.europa.eu/api/v5_2/DRcalc'

api_headers = {'Accept': 'application/json'}

usf_ana1_params = {
    'lat': -1.340332,
    'lon': -48.368021,
    'area_modulo': 2.012016,
    'nro_modulos': 328,
    'eficiencia_modulo': 0.1988,
    'capacidade': 131.2,
    'device_id': 1
}

usf_sat_params = {
    'lat': -1.071451,
    'lon': -48.126467,
    'area_modulo': 2.556048,
    'nro_modulos': 204,
    'eficiencia_modulo': 0.215,
    'capacidade': 111.18,
    'device_id': 14
}

usf_buj_params = {
    'lat': -1.667214,
    'lon': -48.054720,
    'area_modulo': 2.578716,
    'nro_modulos': 612,
    'eficiencia_modulo': 0.2055,
    'capacidade': 324.36,
    'device_id': 0
}

usf_ana2_params = {
    'lat': -1.340250,
    'lon': -48.368181,
    'area_modulo': 2.173572,
    'nro_modulos': 228,
    'eficiencia_modulo': 0.207,
    'capacidade': 102.6,
    'device_id': 3
}

api_params = {
    'lat': -1.071451,
    'lon': -48.126467,
    'month': 9,
    'global': 1,
    'raddatabase': 'PVGIS-SARAH2',
    'localtime': 1,
    'angle': 10,
    'aspect': 180,
    'showtemperatures': 1,
    'outputformat': 'json'
}

try:
    res = req.get(api_url, params=api_params, headers=api_headers)
    res.raise_for_status()
    json_res = res.json()
    try:
        with open("sample.json", "w") as out_file:
            data = json.dump(json_res, out_file)
    except Exception as e:
        print(f"(ERRO) ao processar JSON: {e}")

except req.exceptions.RequestException as e:
    raise SystemExit(e)

lista = encontrar_chave(json_res, 'daily_profile')
try:
    lista_filtrada = [{chave['time']: chave['Gd(i)']}
                    for chave in lista]
except KeyError:
    print("(ERRO) Algumas chaves não estão presentes nos dicionários.")

chave_inicio = '06:00'
chave_fim = '18:00'

# Verificar e remover as chaves antes e depois das chaves de interesse
inicio_encontrado = False
nova_lista = []
for dicionario in lista_filtrada:
    if chave_inicio in dicionario:
        inicio_encontrado = True
    if inicio_encontrado:
        nova_lista.append(dicionario)
        if chave_fim in dicionario:
            break
del lista, lista_filtrada
print(type(nova_lista[0]['06:00']))

In [ ]:
from datetime import datetime

format_time = '%m-%H:%M'
b = 9
time_string = str(b) + '-'

a = '17:00'
a = time_string + a
print(a)

a_datetime = datetime.strptime(a,format_time)


print(a_datetime)
print(a_datetime.month)

In [ ]:
usf_ana1_params = {
    'lat': -1.340332,
    'lon': -48.368021,
    'area_modulo': 2.012016,
    'nro_modulos': 328,
    'eficiencia_modulo': 0.1988,
    'capacidade': 131.2,
    'device_id': 1
}

usf_sat_params = {
    'lat': -1.071451,
    'lon': -48.126467,
    'area_modulo': 2.556048,
    'nro_modulos': 204,
    'eficiencia_modulo': 0.215,
    'capacidade': 111.18,
    'device_id': 14
}

usf_buj_params = {
    'lat': -1.667214,
    'lon': -48.054720,
    'area_modulo': 2.578716,
    'nro_modulos': 612,
    'eficiencia_modulo': 0.2055,
    'capacidade': 324.36,
    'device_id': 0
}

usf_ana2_params = {
    'lat': -1.340250,
    'lon': -48.368181,
    'area_modulo': 2.173572,
    'nro_modulos': 228,
    'eficiencia_modulo': 0.207,
    'capacidade': 102.6,
    'device_id': 3
}

dicionario = {'USF_ANA1': usf_ana1_params, 'USF_SAT': usf_sat_params, 'USF_BUJ': usf_buj_params, 'USF_ANA2': usf_ana2_params}
del usf_ana1_params, usf_sat_params, usf_buj_params, usf_ana2_params 


In [ ]:
chave_inicio = '06:00'
chave_fim = '18:00'

# Verificar e remover as chaves antes e depois das chaves de interesse
inicio_encontrado = False
nova_lista = []
for dicionario in lista:
    if chave_inicio in dicionario:
        inicio_encontrado = True
    if inicio_encontrado:
        nova_lista.append(dicionario)
        if chave_fim in dicionario:
            break